In [1]:
import pandas as pd 
import numpy as np
import json

import os
from collections import Counter

# Append the entire repo parent director so files therein can be accessed in notebook
import sys
import pathlib
sys.path.append(str(pathlib.Path().absolute().parent))

from src import helper
%load_ext autoreload

### Import Data

In [3]:
county_ts = pd.read_pickle('../data/interim/county-time-series-with-fipsmap.pickle')
county_xwalk = pd.read_csv('../data/interim/CountyCrossWalk_Zillow.csv')

fips_mapping = pd.read_pickle('../data/interim/fips-county-metro-state-df.pickle')

In [14]:
county_xwalk.head(3)

,CountyName,StateName,StateFIPS,CountyFIPS,MetroName_Zillow,CBSAName,CountyRegionID_Zillow,MetroRegionID_Zillow,FIPS,CBSACode
0,Pike,Pennsylvania,42,103,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",280,394913.0,42103,35620.0
1,Bronx,New York,36,5,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",401,394913.0,36005,35620.0
2,Essex,New Jersey,34,13,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",504,394913.0,34013,35620.0


In [17]:
len(set(county_ts['RegionName']).intersection(set(county_xwalk['FIPS'])))

2701

In [5]:
county_ts.head()

,Date,RegionName,DaysOnZillow_AllHomes,InventorySeasonallyAdjusted_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,...,ZHVI_TopTier,ZRI_AllHomes,ZRI_AllHomesPlusMultifamily,ZriPerSqft_AllHomes,Zri_MultiFamilyResidenceRental,Zri_SingleFamilyResidenceRental,CountyName,StateName,MetroName,CensusRegion
0,1996-04-30,10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,129700.0,NaN,NaN,NaN,NaN,NaN,Kent,Delaware,"Dover, DE",South
1,1996-04-30,10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,181400.0,NaN,NaN,NaN,NaN,NaN,New Castle,Delaware,"Philadelphia, PA",South
2,1996-04-30,1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,192300.0,NaN,NaN,NaN,NaN,NaN,Baldwin,Alabama,"Daphne, AL",South
3,1996-04-30,1009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,123500.0,NaN,NaN,NaN,NaN,NaN,Blount,Alabama,"Birmingham, AL",South
4,1996-04-30,1017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Chambers,Alabama,"Valley, AL",South


In [13]:
county_ts[county_ts['CountyName'].str.contains("Greene")].sort_values(by='StateName')

,Date,RegionName,DaysOnZillow_AllHomes,InventorySeasonallyAdjusted_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,...,ZHVI_TopTier,ZRI_AllHomes,ZRI_AllHomesPlusMultifamily,ZriPerSqft_AllHomes,Zri_MultiFamilyResidenceRental,Zri_SingleFamilyResidenceRental,CountyName,StateName,MetroName,CensusRegion
213541,2007-04-30,5055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,134100.0,NaN,NaN,NaN,NaN,NaN,Greene,Arkansas,"Paragould, AR",South
73080,2000-03-31,5055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,120600.0,NaN,NaN,NaN,NaN,NaN,Greene,Arkansas,"Paragould, AR",South
271866,2010-01-31,5055,NaN,190.0,184.0,NaN,NaN,NaN,NaN,NaN,...,147800.0,NaN,NaN,NaN,NaN,NaN,Greene,Arkansas,"Paragould, AR",South
422159,2014-12-31,5055,NaN,270.0,283.0,NaN,NaN,80.555556,NaN,NaN,...,155400.0,826.0,821.0,0.586,821.0,822.0,Greene,Arkansas,"Paragould, AR",South
71483,2000-02-29,5055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,122100.0,NaN,NaN,NaN,NaN,NaN,Greene,Arkansas,"Paragould, AR",South
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101993,2001-09-30,51079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,227100.0,NaN,NaN,NaN,NaN,NaN,Greene,Virginia,"Charlottesville, VA",South
372792,2013-05-31,51079,NaN,192.0,198.0,NaN,NaN,127.618613,NaN,NaN,...,291300.0,1240.0,1280.0,0.890,NaN,1280.0,Greene,Virginia,"Charlottesville, VA",South
103607,2001-10-31,51079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,226200.0,NaN,NaN,NaN,NaN,NaN,Greene,Virginia,"Charlottesville, VA",South
385750,2013-10-31,51079,NaN,214.0,229.0,NaN,NaN,128.539220,NaN,NaN,...,284300.0,1278.0,1303.0,0.918,NaN,1304.0,Greene,Virginia,"Charlottesville, VA",South


In [6]:
county_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518791 entries, 0 to 518790
Data columns (total 86 columns):
 #   Column                                                         Non-Null Count   Dtype  
---  ------                                                         --------------   -----  
 0   Date                                                           518791 non-null  object 
 1   RegionName                                                     518791 non-null  int64  
 2   DaysOnZillow_AllHomes                                          43692 non-null   float64
 3   InventorySeasonallyAdjusted_AllHomes                           177024 non-null  float64
 4   InventoryRaw_AllHomes                                          177024 non-null  float64
 5   MedianListingPricePerSqft_1Bedroom                             9432 non-null    float64
 6   MedianListingPricePerSqft_2Bedroom                             66693 non-null   float64
 7   MedianListingPricePerSqft_3Bedroom             